In [ ]:
import pandas as pd
import matplotlib

In [ ]:
%run data.ipynb
%run ../modules/preprocessing.ipynb
%run ../modules/cds.ipynb
%run ../modules/utils.ipynb

In [ ]:
df27 = load_uc2(27)
df28 = load_uc2(28)
df27.shape, df28.shape

In [ ]:
df_src = df27.copy()
# df_src = df_chosen.copy()
plotly_zone(df_src)

In [ ]:
# direct sequencing

low = 5500
high = 8500
full = 88*318
# low, high = full - high, full - low
print(low, high)
df_top = df_src[(df_src.Mass>low)&(df_src.Mass<high)].sort_values('Vol', ascending=False).iloc[:200].copy()
# df_top = df_src[(df_src.Mass>low)&(df_src.Mass<high)].copy()
# df_top = df_src[(df_src.Mass>full-high)&(df_src.Mass<full-low)].sort_values('Vol', ascending=False).iloc[:200].copy()
# df_top = df_src.copy()
plotly_zone(df_top)

# df_salts = list()
# for salt in [21.9819, 37.9558, 34.9694, 27.9949, 56.92]:
#     df_salt = match_dfs(df_top, df_top, shift=salt)
# #     df_top = df_top.drop(df_salt.index)
#     df_salts.append(df_salt)
# df_salts = pd.concat(df_salts)
# # plotly_zones(df_top, df_salts)

bcr = base_calling_random(df_top)
plotly_basecalling(*bcr, annotate=False)
# groups = basecalling_groups(df_top)
# plotly_basecalling(*groups[0], annotate=False)

dfm = match_dfs(bcr[0], df_src, shift=-1*M)
if not dfm.empty:
    plotly_zones(bcr[0], dfm)

In [ ]:
# Choose dots

idxs = list()
# create our callback function
def on_selection(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

def on_click(trace, points, selector):
#     print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

# df_sample = df_src[(df_src.Mass>10000)&(df_src.Mass<27000)]
# df_sample = df_top
df_sample = df_chosen
f = zone_selection(df_sample, on_selection=on_selection, on_click=on_click)
f

In [ ]:
df_chosen = df_sample.iloc[idxs]
# df_chosen = df_sample.drop(df_chosen.index)
df_chosen.shape

In [ ]:
df_sample = df_chosen.sort_values('Vol', ascending=False).iloc[:50]
# df_sample = df_sample[(df_sample.Mass>3000)&(df_sample.Mass<4800)]
# df_sample = df_chosen
orientation = 5

# df_sf, df_salts = remove_salts(df_sample)
df_sf = df_sample

bcr = base_calling_random(df_sf)
plotly_basecalling(*bcr, annotate=False, y='RT')
group = bcr

# groups = basecalling_groups(df_sf)
# group = groups[0]
# plotly_basecalling(*group, annotate=False)

# output seats
# df = process_mass_base_seats(*group, full_mass=24174, orientation=orientation)
# df.to_excel('~/Downloads/seats.xlsx')


dfm = match_dfs(bcr[0], df_src, shift=-1*M)
if not dfm.empty:
    plotly_zones(bcr[0], dfm, y='RT')
else:
    plotly_zone(bcr[0])

In [ ]:
target = 7259
dfa = df_src[(df_src.Mass>target-1)&(df_src.Mass<target+1)]
target += M
dfb = df_src[(df_src.Mass>target-1)&(df_src.Mass<target+1)]
dfa, dfb

In [ ]:
# bcr[0].to_excel('~/Downloads/type_position_ori.xlsx')
dfa = bcr[0].copy()
dfb = dfm.copy()
dfa['DeMethyl'] = False
dfb['DeMethyl'] = True
dfab = pd.concat([dfa, dfb])
dfab.to_excel('~/Downloads/type_position_ori.xlsx')

In [ ]:
def base_calling_random(df, silence=False, homology=False, acid_labile=False, methyl=False):
    df = df.sort_index()
    mass_x = np.array(df.Mass)
    mass_y = np.array(df.Mass)
    if acid_labile:
        mass_x = np.array(df[df.isAD == False].Mass)
        mass_y = np.array(df[df.isAD == True].Mass)
    if not silence:
        print("mass_x {} mass_y {}".format(mass_x.shape, mass_y.shape))

#     base_dict = {'C': 305.0413, 'A': 329.0525, 'G': 345.0474, 'U': 306.0253, 'g': 359.0631, 'D': 308.041, 'c': 319.0569, 'u': 320.041, 'G^': 373.0787, 'a': 343.0682, 'y': 212.0086, 'Y': 570.1475, 'P': 557.2251, 'x': 688.1156, 'z': 625.0844, 'I': 330.03654, 'O': 344.052}
    base_dict = {'C': 305.0413, 'A': 329.0525, 'G': 345.0474, 'U': 306.0253, 
                 'mG': 359.0631, 'D': 308.041, 'mC': 319.0569, 
                 'mU': 320.041, 'mA': 343.0682#, 'm22C': 333.0625,
#                  'ac4C': 347.05185
                #'mcm5U': 378.0464
                } #, 'y': 212.0086, 'mnm5s2U': 365.04466, 'X': 449.08299, 's4U': 322.00246, 'ms2io6A': 459.09776
#     base_dict = {'g': 359.0631, 'u': 320.041, 'mnm5s2U': 365.04466, 'X': 449.08299, 's4U': 322.00246, 'ms2io6A': 459.09776} #387.1273+61.9557
    if homology:
#         base_dict = {'Me': 14.0106} 
#         base_dict = {'C (305.04)': 305.0413, 'A (329.05)': 329.0525, 'endA (249.09)': 249.0862, 'endC (225.08)': 225.075, 'Me (14.01)': 14.0106, '2Me (14.01)': 28.0212} #, 'Udiff': 43
        base_dict = {'C': 305.0413, 'A': 329.0525, 'Ai': 249.0862, 'Ci': 225.075, 'Ox': 16.0, 'Methyl': 14.0106, '2Methyl': 28.0212} #, 'Udiff': 43
#         base_dict = {'Y (358.16)': 358.1599, 'C (305.04)': 305.0413, 'A (329.05)': 329.0525, 'endA (249.09)': 249.0862, 'endC (225.08)': 225.075, 'Methyl (14.01)': 14.0106}
    if acid_labile:
#         base_dict = {'Y': 358.1599, 'm6t6A': 276.09568, 'Gr(p)': 345.04602, 'cnm5U': 133.0262, 'I': 118.02654, 'g6A': 218.05381, 'o2yW': 390.12737, 'ms2t6A': 308.06775, 'acp3U/cmnm5Um': 195.06298, 'mcmo5U': 182.03135}
        base_dict = {'Y': 358.1599}
    if methyl:
        base_dict = {'Me': 14.0106} #, '2Me': 28.0212
    pairs = list()
    idxs = list()
    PPM = 10
    for k in base_dict.keys():
        base_mass = base_dict.get(k)
        ppm_matrix = np.abs((mass_x[:, np.newaxis] - mass_y - base_mass) * 1E6 / (mass_y + base_mass))
        ppm_df = pd.DataFrame(ppm_matrix)
        crosstalk = ppm_df[ppm_df < PPM]
        idx_pairs = list(crosstalk[crosstalk.notnull()].stack().index)
        if idx_pairs:
            idx_pairs = [(*pair, k) for pair in idx_pairs] # append base name into idx_pairs
            pairs.extend(idx_pairs)
        df3_idxs = [pair[0] for pair in idx_pairs]
        df5_idxs = [pair[1] for pair in idx_pairs]
        df3_idxs = list(set(df3_idxs))
        df5_idxs = list(set(df5_idxs))
#         if not silence:
#             print("df3_idxs {} df5_idxs {}".format(len(df3_idxs), len(df5_idxs)))
        if df3_idxs:
            idxs.extend(df3_idxs)
        if df5_idxs:
            idxs.extend(df5_idxs)
    
    mass_pairs = [(df.Mass.iloc[p[0]], df.Mass.iloc[p[1]], p[2]) for p in pairs]
    if acid_labile:
        mass_pairs = [(df[df.isAD == False].Mass.iloc[p[0]], df[df.isAD == True].Mass.iloc[p[1]], p[2]) for p in pairs]
    mass_pairs = [mp for mp in mass_pairs if mp[0] != mp[1]] # remove item which contains duplicated values
    idxs = list(set(idxs))
#     plt.figure(figsize=(16, 12))
    df_base_calling = df.iloc[idxs]
    if acid_labile:
        masses = [pair[0] for pair in mass_pairs]
        masses_1 = [pair[1] for pair in mass_pairs]
        masses.extend(masses_1)
        masses = list(set(masses))
        df_base_calling = df[df.Mass.isin(masses)]
    return df_base_calling, mass_pairs

In [ ]:
com